# Сортировка и анализ карьерных траекторий в резюме

In [3]:
import os, glob
import pickle
import pymorphy2
import nltk
import numpy as np
import re
from typing import List
import collections
import datetime
from cytoolz import pipe
from tqdm import tqdm

In [4]:
resume = pickle.load(open("resume.pck", "rb"))
vacancy = pickle.load(open("vacancy.pck", "rb"))

In [ ]:
#word2vec
morph = pymorphy2.MorphAnalyzer()
stopwords = nltk.corpus.stopwords.words(
    'russian') + nltk.corpus.stopwords.words('english')
stopwords += [
    'отличный', 'метр', 'наш', 'клиент', 'банка', 'проект', 'литр',
    'желательный', 'др', 'самый', 'мочь', 'хороший', 'год', 'чел', 'обязательный'
]

cache = {}


def remove_numbers(text):
    return re.sub(r'\d+', '', text)


def _get_POS(word):
    if word not in cache:
        cache[word] = morph.parse(word)[0]
    return cache[word].tag.POS


def normal_form(word):
    if word not in cache:
        cache[word] = morph.parse(word)[0]
    return cache[word].normal_form


def is_word_pos_in(word: str, pos: List[str] = None) -> bool:
    if not pos:
        pos = ['NOUN', "ADJF", 'INFN', 'VERB', 'ADJS']

    return _get_POS(word) in pos


def get_words(text):
    return re.findall(r'\w+', text)


def nonempty(x):
    if isinstance(x, Sequence):
        return filter(lambda x: len(x) > 0 and x != ' ', x)
    return x

helper = {}


def remove_numbers(text):
    return re.sub(r'\d+', '', text)


def normalize_skill(skill: str):
    parsed = tuple(
        pipe(
            skill,
            lambda x: x.lower(),
            remove_numbers,
            get_words,
        ))
    
    clear_skill = []
    dirty_skill = []
    
    # Последнее стоп слово для dirty_skill
    last_stopword = None
    
    # Для каждого слова в скилле
    for i in parsed:
        # Нормализуем слово
        word = normal_form(i)
        
        # Если стоп слово - запомним его
        if word in nltk.corpus.stopwords.words('russian'):
            last_stopword = word
            
            if word == "без":
                clear_skill.append(word)
        
        # Проверим на часть речи, длинну и стоплова
        elif is_word_pos_in(word) and len(word) > 3 and word not in stopwords:
            
            # Если до этого было стоп слово, добавим его в dirty
            if last_stopword and len(dirty_skill) > 0:
                dirty_skill.append(last_stopword)
                last_stopword = None
            
            # Добавим в чистый скилл слово
            clear_skill.append(word)
            
            if is_word_pos_in(word, ['NOUN', 'ADJF']):
                dirty_skill.append(i)
    #f len(clear_skill) > 1 and len(clear_skill) < 8:
    return clear_skill, dirty_skill

In [ ]:
tmp_vector = np.array([0] * 300, dtype=np.float32)
def _word2vec(word):
    for i in ["_NOUN", "_ADJ", "_VERB"]:
        tmp = "{}{}".format(word, i)
        if tmp in model:
            return model[tmp]
        else:
            return tmp_vector
skill_to_vec = lambda x: np.mean(list(map(_word2vec, x)))

In [8]:
#заполняем пропущенные данные и считаем количество дней, которое проработал человек на каждой работе
def prepare_resumes():
    for res in tqdm(resume):
        if 'work_history' in res.keys():
            for work in res['work_history']:
                
                if(work['yearend'] in [None, '0']):
                    work['yearend'] = datetime.datetime.now().strftime("%Y")
                    work['monthend'] = datetime.datetime.now().strftime("%m")
                
                if(work['yearbeg'] == None or work['yearbeg'] == '0'):
                    work['yearbeg'] = work['yearend']
                    
                if(work['monthbeg'] == '0'):
                    work['monthbeg'] = '1'
                
                if(work['monthend'] == '0'):
                    work['monthend'] = '12'
                #print(work['monthbeg'] + " " + work['monthend'] + " " + work['yearbeg'] + " " + work['yearend'])
                #print()
                start = datetime.datetime.strptime(work['monthbeg']+work['yearbeg'], "%m%Y").date()
                finish = datetime.datetime.strptime(work['monthend']+work['yearend'], "%m%Y").date()
                work['work_days'] = (finish-start+datetime.timedelta(days=1)).days

In [43]:
#список всех работ человека, отсортированный по времени
def get_jobs(resume_id):
    ans = {}
    for job in resume[resume_id]['work_history']:
        ans[int(job['monthbeg'])/12 + int(job['yearbeg'])] = job
    return dict(collections.OrderedDict(sorted(ans.items())))

In [9]:
prepare_resumes()

100%|██████████| 128945/128945 [00:30<00:00, 4197.96it/s]


In [10]:
a = 1
for res in resume:
    for work in res['work_history']:
        work['id'] = a
        a += 1

In [11]:
pickle.dump(resume, open("resume_with_days.pkl", 'wb'))

In [104]:
resume[0]

{'additional_education': [],
 'age': 40,
 'base_education': [{'education_form': 'Дневная/Очная',
   'education_type': 'Высшее',
   'faculty': 'Экономический факультет',
   'institute': 'Московский государственный университет им. М.В. Ломоносова',
   'monthend': '0',
   'profession': 'Специализация "Экономика фирмы и отраслевых рынков" (магистр)',
   'town': 'Москва',
   'yearend': '2001'},
  {'education_form': 'Дневная/Очная',
   'education_type': 'Высшее',
   'faculty': 'Экономический факультет',
   'institute': 'Московский государственный университет им. М.В. Ломоносова',
   'monthend': '0',
   'profession': 'Общая экономика (бакалавр)',
   'town': 'Москва',
   'yearend': '1999'}],
 'best': 'Компьютерные навыки:\nMS Excel 2007 (продвинутый пользователь, в т.ч. сводные таблицы), MS Word 2007 (продвинутый пользователь), MS PowerPoint 2007 (уверенный пользователь), MS Access 2007, 1С-Предприятие 7.7, 8.1 (уверенный пользователь), Справочная правовая система КонсультантПлюс 3000, Электро

In [105]:
get_jobs(0)

{1997.1666666666667: {'company_scope': None,
  'id': 9,
  'is_last': '0',
  'monthbeg': '2',
  'monthend': '4',
  'name': '336d370779b3261dcb00331fba5f7f47',
  'profession': 'Менеджер по клиентским отношениям',
  'town': 'Москва',
  'type': 'Полная занятость',
  'work': 'Ведение переговоров с потенциальными клиентами банка по поводу предоставления банковских услуг. ',
  'work_days': 60,
  'work_months': '2.0',
  'yearbeg': '1997',
  'yearend': '1997'},
 1997.5: {'company_scope': None,
  'id': 8,
  'is_last': '0',
  'monthbeg': '6',
  'monthend': '11',
  'name': '49748b69e3d3435cd98d241a6935f3d1',
  'profession': 'Экономист',
  'town': 'Москва',
  'type': 'Полная занятость',
  'work': 'Ведение складского учёта на ПК. ',
  'work_days': 154,
  'work_months': '5.133333333333334',
  'yearbeg': '1997',
  'yearend': '1997'},
 2000.0833333333333: {'company_scope': None,
  'id': 7,
  'is_last': '0',
  'monthbeg': '1',
  'monthend': '6',
  'name': '7119f26423d0e892d1111d8a49372b15',
  'professio